# Sentimen Analisis Report Eng

reporting hasil training model dengan acc score dan f1 score

In [1]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.models import model_from_json
from keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


### load dataset

In [2]:
print("reading")
df = pd.read_csv('lib/clean_text.csv',index_col=0)
df['target']=df['target'].map({0:-1,1:0,2:1})
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df = shuffle(df)

reading


/root/anaconda3/envs/py36/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### downsampling

In [3]:
dneg=df.query('target == -1')
dnet=df.query('target == 0')
dpos=df.query('target == 1')

minlen=min(len(dnet.index),len(dpos.index),len(dneg.index))

dnet=dnet.head(minlen)
dneg=dneg.head(minlen)
dpos=dpos.head(minlen)

data_training=pd.concat([dneg.text,dnet.text,dpos.text])
target=pd.concat([dneg.target,dnet.target,dpos.target])

target=to_categorical(target-target.min())

### tidak downsampling

In [4]:
data_training=df.text
target=df.target
target=to_categorical(target-target.min())

### tokenizing

In [5]:
print('tokenizing')
tokenizer = Tokenizer(num_words=100000)
with open('lib/tokenizer_eng.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
sequences = tokenizer.texts_to_sequences(data_training) 

tokenizing


### padding

In [6]:
print('padding')
input_seq = pad_sequences(sequences, maxlen=70)

padding


### dataset

In [7]:
input_val=input_seq
target_val=np.array(target)
print("data : ",len(input_val))

data :  1620947


### fungsi load model

In [8]:
def load_model(ledom):
    json_file = open(ledom, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    return model

### fungsi acc score

In [9]:
def score(model,we):
    model.load_weights(we)
    scores = model.evaluate(input_val, target_val)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

### fungsi f1 score

In [10]:
def f1(model):
    predict = model.predict_classes(input_val)
    y_true = np.argmax(target_val, axis=1, out=None)
    y_pred = predict
    target_names = ['negative', 'neutral', 'positive']
    print (classification_report(y_true, y_pred, target_names=target_names))

### fungsi report

In [11]:
def report(model_name,model_path,weights_path, score,f1):
    for x in range(len(model_name)):
        print("\n\n","="*20,model_name[x],"="*20)
        model=load_model(model_path[x])
        if score==True:
            score(model,weights_path[x])
            print()
        if f1==True:
            f1(model)

### set nama model, path model, path weights

In [12]:
model_name=["Word2Vec",
            "GLOVE",
            "DBOW",
            "DMM",
            "DMC",
            "DBOW-DMM",
            "DBOW-DMC",
            "DMM-DMC",
            "DBOW-DMM-DMC"]

model_path=["lib/model_w2v_eng.json",
           "lib/model_glove_eng.json",
           "lib/model_dbow_eng.json",
           "lib/model_dmm_eng.json",
           "lib/model_dmc_eng.json",
           "lib/model_dbow-dmm_eng.json",
           "lib/model_dbow-dmc_eng.json",
           "lib/model_dmm-dmc_eng.json",
            "lib/model_dbow-dmm-dmc_eng.json"]

weights_path=["lib/weights_w2v.hdf5",
           "lib/weights_glove.hdf5",
           "lib/weights_dbow.hdf5",
           "lib/weights_dmm.hdf5",
           "lib/weights_dmc.hdf5",
           "lib/weights_dbow-dmm.hdf5",
           "lib/weights_dbow-dmc.hdf5",
           "lib/weights_dmm-dmc.hdf5",
            "lib/weights_dbow-dmm-dmc.hdf5"]

# report

parameter
1. list nama model
2. list path model
3. list path weight
4. acc score 
5. f1 score

In [13]:
report(model_name,model_path,weights_path,score=True,f1=True)



 ==================== Word2Vec ====================
1620947/1620947 [==============================] - 276s 170us/step
categorical_accuracy: 78.33%



/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

    negative       0.79      0.77      0.78    814047
     neutral       0.00      0.00      0.00      6227
    positive       0.77      0.80      0.79    800673

   micro avg       0.78      0.78      0.78   1620947
   macro avg       0.52      0.52      0.52   1620947
weighted avg       0.78      0.78      0.78   1620947



 ==================== GLOVE ====================
 166048/1620947 [==>...........................] - ETA: 4:13

KeyboardInterrupt: 